# Introduction
In this notebook, I will try to validate the model with the 13C data presented in Tang et al. 2009 (DOI 10.1002/bit.22181).

To do this, I will fix the internal fluxes to match that measured by the 13C data and from there see how much the exchange fluxes match the measured data. 

The paper has data about aerobic and microaerobic 13C data. and also have some data about the mixed acid fermentation. Table 2 will provide most of the experimental data to validate against? But these are yields.. I'll just have to make do.

Also, oxygen consumption rates aren't really defined.. They are more just defined based on the observed growth rate? so I will need to find a way around this.

In the paper, the glucose uptake rate is often set to 5 mmol h-1 g-1 biomass. So I will do the same here to make the values more comparable.

N.B.: this is a bit of a different strain! it is the M10EXG strain. 

To compare the calculated 13C fluxes with our model, I will first summarize them in a data frame. For each reaction defined by the C13-model, i will write the flux (relative to the 100 units shown) and try to map the corresponding flux from our model that it should be set to. 

In [133]:
import cameo
import pandas as pd
import cobra.io
from escher import Builder
import copy
from cobra.flux_analysis import flux_variability_analysis
from cobra import Metabolite, Reaction

In [134]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

# Aerobic condition
I will first look to see if we can validate the aerobic condition that is tested. I will import the data frame from the file made in the 'databases' folder of the repo. Here i coupled the reactions and fluxes shown in the Tang paper to our model.

In [3]:
#set the glucose consumption to 5 mmol/gCDW/h
model.reactions.EX_glc__D_e.bounds = (-5,1000)

In [4]:
model.optimize().objective_value

0.32861628193475195

Now I need to write a script which will fix the fluxes as defined here and then will see if optimization is possible.
Then I will also check that the predicted products are and if this matches the experimental data.

When I do this with the exact fluxes determined, I get an infeasible solution. This is because for many of the reactions the standard deviation isn't given, allowing very little flexibility in the model. For the reactions with standard deviation, we will include them as flexibility in the bounds as: mean +-2xstDev.
For reactions without any standard deviation, we will add an error of 10%. This isn't ideal but may be necessary.

In [191]:
df_aer = pd.read_csv('../databases/Tang 2009 fluxes-aerobic.csv')

In [192]:
#note: aerobic is the fluxes from the paper, without GLUSy (i.e. the flux to glutamate in the biomass)

In [193]:
del df_aer['N.B.: 0 means no info available']

In [194]:
model_copy = copy.deepcopy(model)
for index, row in df_aer.iterrows():
    rct_id = row['model_id']
    rct = model_copy.reactions.get_by_id(rct_id)
    ub = row['UB']/20 #normalize to the 5 mmol glucose that is being supplied
    lb = row['LB']/20 #normalize to the 5 mmol glucose that is being supplied
    rct.bounds = (lb,ub)
    pfba_solution = cobra.flux_analysis.pfba(model_copy)
    print(rct_id,pfba_solution['biomass'])


GLC__Dtpts 0.32861626365989793
GALUi 0.3286162636599081
G6PDH2r 0.3269833815743214
PRPPS 0.2902505656145208
TKT2 0.2902505656145299
TALA 0.2902505656145228
TKT1 0.2902505656144517
DDPA 0.29025056561478846
PGI 0.29025056561478857
FBA 0.29025056554757445
PGK 0.2902505656145884
ENO 0.2902505656145466
PGCD 0.29025056561456464
GHMT 0.29025056561464996


Infeasible: None (infeasible)

We can see that the solver errors after the GHMT reaction. So I will fix all the other reactions and then see what is going on in metabolism to explain where this issue comes from.



In [233]:
model.reactions.GALUi.bounds = (0.045,0.055)
model.reactions.G6PDH2r.bounds = (-1.7,-1.3)
model.reactions.PRPPS.bounds = (0.18,0.22)
model.reactions.TKT2.bounds = (-0.44, -0.36)
model.reactions.TALA.bounds = (0.405,0.495)
model.reactions.TKT1.bounds = (-0.495,-0.405)
model.reactions.DDPA.bounds = (0.09,0.11)
model.reactions.PGI.bounds = (-3.795,-3.105)
model.reactions.FBA.bounds = (3.87,4.73)
model.reactions.PGK.bounds = (-9.35,-8.55)
model.reactions.ENO.bounds = (7.605,9.295)
model.reactions.PGCD.bounds = (0.45,0.55)
model.reactions.GHMT.bounds = (0.315,0.385)


In [234]:
model.reactions.PDH.bounds = (7.515,9.185)
model.reactions.PFL.bounds = (0,0)

In [235]:
model.reactions.LDH_L.bounds = (-0.44,-0.36)
model.reactions.ALCD2x.bounds = (0,0)
model.reactions.ASPTA.bounds = (-0.44,-0.36)
model.reactions.CS.bounds = (-3.5,-2.9)
model.reactions.ICDHyr.bounds = (1.8,2.2)
model.reactions.ICL.bounds = (0.6,1.8)
model.reactions.SUCOAS.bounds = (-2.2,-1.2)
model.reactions.SUCDi.bounds = (2.61,3.19)
model.reactions.FUM.bounds = (2.61,3.19)
model.reactions.MDH.bounds = (-4.51,-3.69)


In [236]:
pfba_solution = cobra.flux_analysis.pfba(model)
pfba_solution['biomass']

0.1796160053852448

In [60]:
aa =['EX_his__L_e', 'EX_lys__L_e', 'EX_met__L_e', 'EX_phe__L_e', 'EX_thr__L_e', 'EX_trp__L_e', 'EX_asp__L_e', 'EX_asn__L_e', 'EX_ala__L_e', 'EX_arg__L_e','EX_cys__L_e', 'EX_gln__L_e', 'EX_glu__L_e', 'EX_gly_e', 'EX_pro__L_e','EX_ser__L_e', 'EX_tyr__L_e','EX_ile__L_e', 'EX_leu__L_e','EX_val__L_e']

In [238]:
#Are we producing any amino acids???
for acid in aa:
    if pfba_solution[acid] > 0:
        print(acid,pfba_solution[acid])
    else: continue


EX_phe__L_e 0.026745818132085496
EX_pro__L_e 0.04599100831919913


So, we are producing some proline and phenylalanine, but it is not so much so I think it is oke for now.

In this setting I've fit all fluxes except the PSCVT, PPCK and PC reactions. (And GLUSy and PTAr, as we decided).
Now I need to figure out what the cause is of why I can't fix these fluxes?

First: what are the fluxes in the best fit when all other fluxes are fit?

In [239]:
print('PSCVT',pfba_solution['PSCVT'])
print('PPCK',pfba_solution['PPCK'])
print('PC',pfba_solution['PC'])


PSCVT 0.09
PPCK 0.0
PC 0.0


In [240]:
model.reactions.PC.bounds = (0,5.3)
model.reactions.PPCK.bounds = (0,6.5)

So the only flux that I can't fix is that of PSCVT, which matches the reaction that produces phenylalanine. Maybe it should be fit to a different reaction in the model? 

It was fit to the PSCVT, which is the first brance to the shikimate pathway. Maybe I should fix it to a different reaction further downstream? When it is fit to PHETA1 it can be solved... this is unexpected but it does seem to work.

In [241]:
model.reactions.PHETA1.bounds = (-0.22,-0.18)

In [242]:
pfba_solution = cobra.flux_analysis.pfba(model)
pfba_solution['biomass']

0.17961600538522435

In [243]:
for acid in aa:
    if pfba_solution[acid] > 0:
        print(acid,pfba_solution[acid])
    else: continue

EX_phe__L_e 0.026745818132092675
EX_pro__L_e 0.045991008319333074


When the two changes above are made in the table, it should solve the issue.

In [135]:
df_aer = pd.read_csv('../databases/Tang 2009 fluxes-aerobic.csv')

To be able to fit the glutamate being pulled into biomass, we need to add a reaction to the model. Because the GLUSy reaction that forms glutamate is also used in the formation of glutamine, so fixing that would be incorrect (and infeasible). 

To do so, I will add a new metabolite that can pull a specific amount from the glutamate pool, and from there go into biomass.

In [136]:
model_copy = copy.deepcopy(model)

In [137]:
#add metabolite
model_copy.add_metabolites(Metabolite(id='glu__L_c_biomass'))

In [138]:
model_copy.metabolites.glu__L_c_biomass.formula = model_copy.metabolites.glu__L_c.formula

In [139]:
#add pulling reaction
model_copy.add_reaction(Reaction(id='glu_biomass'))

In [140]:
model_copy.reactions.glu_biomass.add_metabolites({
    model_copy.metabolites.glu__L_c:-1,
    model_copy.metabolites.glu__L_c_biomass:1
})

In [141]:
#change biomass consumption to the new metabolites
model_copy.reactions.biomass.add_metabolites({
    model_copy.metabolites.glu__L_c:0.322004464285714,
    model_copy.metabolites.glu__L_c_biomass:-0.322004464285714
})

In [142]:
#also need to add the exchange reaction incase the flux doesn't match the defined aa composition in the biomass reaction
#it would error otherwise.
model_copy.add_boundary(model_copy.metabolites.glu__L_c_biomass,type = 'sink', reaction_id = 'glu_sink')

Reaction identifier,glu_sink
Name,sink
Memory address,0x028515ab0108
Stoichiometry,glu__L_c_biomass <=> <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [143]:
for index, row in df_aer.iterrows():
    rct_id = row['model_id']
    rct = model_copy.reactions.get_by_id(rct_id)
    ub = row['UB']/20 #normalize to the 5 mmol glucose that is being supplied
    lb = row['LB']/20 #normalize to the 5 mmol glucose that is being supplied
    rct.bounds = (lb,ub)
    pfba_solution = cobra.flux_analysis.pfba(model_copy)
    print(rct_id,pfba_solution['biomass'])


GLC__Dtpts 0.6645612560348155
GALUi 0.6645612560348276
G6PDH2r 0.6628986507031949
PRPPS 0.2902505656145846
TKT2 0.29025056561458296
TALA 0.2902505656145725
TKT1 0.29025056561455553
PHETA1 0.29025056561450047
PGI 0.2902505656145693
FBA 0.290250565614613
PGK 0.29025056561458085
ENO 0.2902505656146166
PGCD 0.2902505656146604
GHMT 0.2902505656145413
PPCK 0.2902505656145413
PC 0.2902505656145413
PDH 0.2902505656145147
PFL 0.29025056561451174
LDH_L 0.29025056561454715
ALCD2x 0.2902505656146235
ASPTA 0.17961600538523312
CS 0.17961600543260142
ICDHyr 0.1796160053852298
ICL 0.17961600538521627
SUCOAS 0.17961600538523062
SUCDi 0.17961600538522118
FUM 0.17961600538523945
MDH 0.1796160053852237
PTAr 0.17961600538520706
glu_biomass 0.17961600538518965


Now to check which amino acids are being made in overflow from the defined fluxes

In [144]:
pfba_solution = cobra.flux_analysis.pfba(model_copy)
pfba_solution['biomass']

0.17961600538518965

In [145]:
for acid in aa:
    if pfba_solution[acid] > 0:
        print(acid,pfba_solution[acid])
    else: continue

So we seem to not be producing any amino acids in excess, butwe do have quite a bit of glutamate that is being produced too much..

In [146]:
model_copy.optimize()['glu_sink']

0.21216284440880212

Now that the fluxes have been fit, we would want to see what the predicted exchange rates are and then compare this to the published data.


In [147]:
prod = ['EX_glc__D_e', 'EX_lac__L_e','EX_etoh_e', 'EX_ac_e', 'EX_for_e']

In [148]:
for comp in prod:
    pfba_solution = cobra.flux_analysis.pfba(model_copy)
    print(comp,':  ',pfba_solution[comp]/pfba_solution['EX_glc__D_e'])

EX_glc__D_e :   1.0
EX_lac__L_e :   -0.07475398074528825
EX_etoh_e :   -0.0
EX_ac_e :   -0.7731389661810618
EX_for_e :   -0.03659758425579478


In [150]:
for comp in prod:
    print( comp, ':  ', flux_variability_analysis(model_copy, model_copy.reactions.get_by_id(comp)))

EX_glc__D_e :                 minimum   maximum
EX_glc__D_e -7.037492 -4.558571
EX_lac__L_e :                minimum   maximum
EX_lac__L_e     0.36  3.339142
EX_etoh_e :              minimum  maximum
EX_etoh_e      0.0      0.0
EX_ac_e :            minimum   maximum
EX_ac_e      0.0  4.787058
EX_for_e :             minimum   maximum
EX_for_e      0.0  0.497233


In [151]:
#what is the oxygen consumption rate here?

In [152]:
pfba_solution = cobra.flux_analysis.pfba(model_copy)
pfba_solution['EX_o2_e']

-12.554586813012193

In [153]:
flux_variability_analysis(model_copy, model_copy.reactions.EX_o2_e)

,minimum,maximum
EX_o2_e,-16.18879,-11.060006


## Microaerobic
Here an unfortunate thing is that we lack the information about the amount of oxygen that is supplied for the microaerobic experiment. So what i will do is fit the fluxes as they are defined, and then see what oxygen consumption the model predicts. 

__Alternatively, I set the oxygen supply to an arbitrarily selected 'micro' aerobic system....__ ??


In [154]:
#set the glucose consumption to 5 mmol/gCDW/h
model.reactions.EX_glc__D_e.bounds = (-5,1000)

In [155]:
df_micaer = pd.read_csv('../databases/Tang 2009 fluxes-microaerobic.csv')

In [157]:
del df_micaer['N.B.: 0 means no info available']

In [158]:
model_copy = copy.deepcopy(model)

In [159]:
#add metabolite
model_copy.add_metabolites(Metabolite(id='glu__L_c_biomass'))

In [160]:
model_copy.metabolites.glu__L_c_biomass.formula = model_copy.metabolites.glu__L_c.formula

In [161]:
#add pulling reaction
model_copy.add_reaction(Reaction(id='glu_biomass'))

In [162]:
model_copy.reactions.glu_biomass.add_metabolites({
    model_copy.metabolites.glu__L_c:-1,
    model_copy.metabolites.glu__L_c_biomass:1
})

In [163]:
#change biomass consumption to the new metabolites
model_copy.reactions.biomass.add_metabolites({
    model_copy.metabolites.glu__L_c:0.322004464285714,
    model_copy.metabolites.glu__L_c_biomass:-0.322004464285714
})

In [164]:
#also need to add the exchange reaction incase the flux doesn't match the defined aa composition in the biomass reaction
#it would error otherwise.
model_copy.add_boundary(model_copy.metabolites.glu__L_c_biomass,type = 'sink', reaction_id = 'glu_sink')

Reaction identifier,glu_sink
Name,sink
Memory address,0x028511897b48
Stoichiometry,glu__L_c_biomass <=> <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [165]:
for index, row in df_micaer.iterrows():
    rct_id = row['model_id']
    rct = model_copy.reactions.get_by_id(rct_id)
    ub = row['UB']/20 #normalize to the 5 mmol glucose that is being supplied
    lb = row['LB']/20 #normalize to the 5 mmol glucose that is being supplied
    rct.bounds = (lb,ub)
    pfba_solution = cobra.flux_analysis.pfba(model_copy)
    print(rct_id,pfba_solution['biomass'])

GLC__Dtpts 0.33447850475472396
GALUi 0.33447850475472357
G6PDH2r 0.3333918050290294
PRPPS 0.14512528280727868
TKT2 0.14512528280728132
TALA 0.14512528280727893
TKT1 0.14512528280726283
PHETA1 0.14512528280725479
PGI 0.1451252828072909
FBA 0.14512528280728706
PGK 0.14512528280729364
ENO 0.1451252828072815
PGCD 0.1451252828072797
GHMT 0.14512528280725104
PPCK 0.14512528280729642
PC 0.1451252828073098
PDH 0.1451252828073098
PFL 0.1451252828073098
LDH_L 0.1451252828072552
ALCD2x 0.1451252828072482
ASPTA 0.11226000336617314
CS 0.11226000336576454
ICDHyr 0.1122600033657635
ICL 0.11226000336574968
SUCOAS 0.11226000336564389
SUCDi 0.1122600033657761
FUM 0.11226000336579965
MDH 0.1122600033657547
PTAr 0.11226000336490721
glu_biomass 0.11226000336576282


In [166]:
pfba_solution = cobra.flux_analysis.pfba(model_copy)
pfba_solution['EX_o2_e']

-5.917379540248659

In [167]:
flux_variability_analysis(model_copy, model_copy.reactions.EX_o2_e)

,minimum,maximum
EX_o2_e,-7.189232,-5.646339


In [168]:
#rati between PFL and PDH?
print(pfba_solution['PDH'])
print(pfba_solution['PFL'])

5.3253215738744855
-0.1770659349476114


In [169]:
prod = ['EX_glc__D_e', 'EX_lac__L_e','EX_etoh_e', 'EX_ac_e', 'EX_for_e']

In [170]:
for comp in prod:
    pfba_solution = cobra.flux_analysis.pfba(model_copy)
    print(comp,':  ',pfba_solution[comp]/pfba_solution['EX_glc__D_e'])

EX_glc__D_e :   1.0
EX_lac__L_e :   -0.6834991258430021
EX_etoh_e :   -0.2621640482685487
EX_ac_e :   -0.454597717646895
EX_for_e :   -0.05548776048569796


In [171]:
for comp in prod:
    print( comp, ':  ', flux_variability_analysis(model_copy, model_copy.reactions.get_by_id(comp)))

EX_glc__D_e :                minimum  maximum
EX_glc__D_e     -5.0 -4.75187
EX_lac__L_e :                minimum   maximum
EX_lac__L_e    3.285  4.057743
EX_etoh_e :              minimum  maximum
EX_etoh_e     1.26     1.54
EX_ac_e :             minimum   maximum
EX_ac_e  0.761025  2.870244
EX_for_e :             minimum   maximum
EX_for_e      0.0  2.491716


## Anaerobic
These fluxes were fit based on the data in figure 3. They are much more limited than the other fluxes. Also they measure the fluxes to all the main fermentation products, so this doesn't really mean the model should differ from the measured data.

I will fix the oxygen rate to be low so we are sure there is no OxPhos activated (-0.5 mmol/gcdw/h).

In [272]:
#set the glucose consumption to 5 mmol/gCDW/h
model.reactions.EX_glc__D_e.bounds = (-5,1000)

In [273]:
df_anaer = pd.read_csv('../databases/Tang 2009 fluxes-anaerobic.csv')

In [274]:
model_copy = copy.deepcopy(model)

In [295]:
#make anaerobic
model_copy.reactions.EX_o2_e.bounds = (-1.5,0)

In [275]:
#add metabolite
model_copy.add_metabolites(Metabolite(id='glu__L_c_biomass'))

In [276]:
model_copy.metabolites.glu__L_c_biomass.formula = model_copy.metabolites.glu__L_c.formula

In [277]:
#add pulling reaction
model_copy.add_reaction(Reaction(id='glu_biomass'))

In [278]:
model_copy.reactions.glu_biomass.add_metabolites({
    model_copy.metabolites.glu__L_c:-1,
    model_copy.metabolites.glu__L_c_biomass:1
})

In [279]:
#change biomass consumption to the new metabolites
model_copy.reactions.biomass.add_metabolites({
    model_copy.metabolites.glu__L_c:0.322004464285714,
    model_copy.metabolites.glu__L_c_biomass:-0.322004464285714
})

In [280]:
#also need to add the exchange reaction incase the flux doesn't match the defined aa composition in the biomass reaction
#it would error otherwise.
model_copy.add_boundary(model_copy.metabolites.glu__L_c_biomass,type = 'sink', reaction_id = 'glu_sink')

Reaction identifier,glu_sink
Name,sink
Memory address,0x02851a4fe448
Stoichiometry,glu__L_c_biomass <=> <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [281]:
for index, row in df_anaer.iterrows():
    rct_id = row['model_id']
    rct = model_copy.reactions.get_by_id(rct_id)
    ub = row['UB']/20 #normalize to the 5 mmol glucose that is being supplied
    lb = row['LB']/20 #normalize to the 5 mmol glucose that is being supplied
    rct.bounds = (lb,ub)
    pfba_solution = cobra.flux_analysis.pfba(model_copy)
    print(rct_id,pfba_solution['biomass'])

PGI 0.32293418705301563
G6PDH2r 0.2939007671953274
GAPD 0.28589207133506167
PFL 0.2858920713350569
PDH 0.28589207133505856
PTAr 0.2732222379248425
ACALD 0.25882807649183803
ALCD2x 0.21425569331962852
LDH_L 0.07615177354497404


In [296]:
pfba_solution = cobra.flux_analysis.pfba(model_copy)
pfba_solution['EX_o2_e']

-1.5

In [297]:
flux_variability_analysis(model_copy, model_copy.reactions.EX_o2_e)

,minimum,maximum
EX_o2_e,-1.5,-1.5


In [298]:
pfba_solution['biomass']

0.0015780710435513384

So the data can fit, but it always requires some oxygen to be able to fuel the OxPhos.. 

The OxPhos is needed to get rid of the quinone formed in the conversion from succinate to fumarate in the reverse TCA. The succoa made is needed for making a precursor in fatty acid synthesis, so we can't completely get rid of it. 

But this is strange, as in our anaerobic analysis this reaction isnt a cause of the oxygen requirement. So in the normal model, anaerobically they can find another way to regenerate qh2. Need to look into what this is and why the model doesn't do that here?

So it turns out that (without fitted data) the model produces succinate via the SSALy reaction, instead of through the TCA. this circumvents the need to regenerate qh2. Why is this not possible with the fitted data?

__Or does this again fit with our argument that the strain cannot grow completely anaerobically? In the experiment there is always a little bit of YE?__

In [299]:
model_copy.metabolites.o2_c.summary()

,,PERCENT,FLUX,REACTION_STRING
RXN_STAT,ID,,,
PRODUCING,O2t,100.000000,1.500000,o2_e <=> o2_c
CONSUMING,CYTBO3,99.996075,1.499941,2.5 h_c + 0.5 o2_c + qh2_c --> h2o_c + 2.5 h_e...


In [322]:
model.reactions.EX_o2_e.bounds = (-0.0005, 0)

In [323]:
model.metabolites.qh2_c.summary()

,,PERCENT,FLUX,REACTION_STRING
RXN_STAT,ID,,,
PRODUCING,AMMQLT,100.0,0.000004,2dmmq_c + amet_c --> ahcys_c + h_c + qh2_c
CONSUMING,biomass,100.0,0.000004,0.000357142857142857 10fthf_c + 0.000249107142...


In [335]:
model.metabolites.get_by_id('agm_c').summary()

,,PERCENT,FLUX,REACTION_STRING
RXN_STAT,ID,,,
PRODUCING,ARGDC,100.0,0.000363,arg__L_c + h_c --> agm_c + co2_c
CONSUMING,AGMT,100.0,0.000363,agm_c + h2o_c --> ptrc_c + urea_c


In [312]:
model_copy.reactions.propACP_c

Reaction identifier,MMSDH
Name,(S)-Methylmalonate semialdehyde:NAD+ oxidoreductase (CoA-propanoylating)
Memory address,0x02851ab16c88
Stoichiometry,coa_c + mmtsa_c + nad_c --> co2_c + nadh_c + ppcoa_c CoA + (S)-Methylmalonate semialdehyde + NAD+ --> CO2 + NADH + Propanoyl-CoA
GPR,RTMO01648 or RTMO01382
Lower bound,0.0
Upper bound,1000.0


In [168]:
#rati between PFL and PDH?
print(pfba_solution['PDH'])
print(pfba_solution['PFL'])

5.3253215738744855
-0.1770659349476114


In [169]:
prod = ['EX_glc__D_e', 'EX_lac__L_e','EX_etoh_e', 'EX_ac_e', 'EX_for_e']

In [170]:
for comp in prod:
    pfba_solution = cobra.flux_analysis.pfba(model_copy)
    print(comp,':  ',pfba_solution[comp]/pfba_solution['EX_glc__D_e'])

EX_glc__D_e :   1.0
EX_lac__L_e :   -0.6834991258430021
EX_etoh_e :   -0.2621640482685487
EX_ac_e :   -0.454597717646895
EX_for_e :   -0.05548776048569796


In [171]:
for comp in prod:
    print( comp, ':  ', flux_variability_analysis(model_copy, model_copy.reactions.get_by_id(comp)))

EX_glc__D_e :                minimum  maximum
EX_glc__D_e     -5.0 -4.75187
EX_lac__L_e :                minimum   maximum
EX_lac__L_e    3.285  4.057743
EX_etoh_e :              minimum  maximum
EX_etoh_e     1.26     1.54
EX_ac_e :             minimum   maximum
EX_ac_e  0.761025  2.870244
EX_for_e :             minimum   maximum
EX_for_e      0.0  2.491716


Note:
- for the micro-aerobic conversion of pyr to acetyl-coa, we need to distinguish PDH from PFL.. for aerobic it is clear it is PDH. For microaerobic, it is hard to discern the exact ratio that one could observe, so i think we should set the bounds to allow the rang of either reaction within the measurement made and from there estimate the ratio of activity between PDH and PFL. 




- maybe i can plot the measured vs simulated fluxes/exchanges or so?


In [300]:
solution = model_copy.optimize()

In [301]:
Builder(model = model_copy, map_json = '../map/g-thermo-map.json', reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'IDPh': 3.452796984101684, 'CAT': 3.699779753750112e-05, 'PDHam1hi': 0.0, 'HYDA': 0.0, …